Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [3]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [4]:
pickle_file = '../notMNIST.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    
    del save  # hint to help gc free up memory
    
    print('Training set', train_dataset.shape, train_labels.shape)
    print('Validation set', valid_dataset.shape, valid_labels.shape)
    print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [68]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
    dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
    # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
    return dataset, labels

train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)

print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [69]:
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
            / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [70]:
batch_size = 128
lmbda = 0.01

graph = tf.Graph()
with graph.as_default():
    
    train = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    
    valid = tf.constant(valid_dataset)
    test = tf.constant(test_dataset)
    lmbda = tf.constant(lmbda)

    w1 = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]))
    b1 = tf.Variable(tf.zeros([num_labels]))
    
    def model(data):
        return tf.matmul(data, w1) + b1

    logits = model(train)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=labels, logits=logits)) \
           + lmbda * tf.nn.l2_loss(weights)

    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(valid))
    test_prediction = tf.nn.softmax(model(test))

ValueError: Tensor("L2Loss_1:0", shape=(), dtype=float32) must be from the same graph as Tensor("Const_2:0", shape=(), dtype=float32).

In [6]:
nb_steps = 3001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        
        feed_dict = {train : batch_data, labels : batch_labels}        
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
    print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 46.549721
Minibatch accuracy: 8.6%
Minibatch loss at step 500: 1.060032
Minibatch accuracy: 78.1%
Minibatch loss at step 1000: 0.795879
Minibatch accuracy: 83.6%
Minibatch loss at step 1500: 0.858160
Minibatch accuracy: 78.1%
Minibatch loss at step 2000: 0.903612
Minibatch accuracy: 75.8%
Minibatch loss at step 2500: 0.722686
Minibatch accuracy: 82.8%
Minibatch loss at step 3000: 0.762428
Minibatch accuracy: 82.8%
Validation accuracy: 81.8%
Test accuracy: 88.4%


It got better!

---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [7]:
k = 5
tiny_train, tiny_labels = train_dataset[:k*batch_size], train_labels[:k*batch_size]

num_steps = 3001
with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        offset = (step * batch_size) % (tiny_labels.shape[0] - batch_size)
        
        batch_data = tiny_train[offset:(offset + batch_size), :]
        batch_labels = tiny_labels[offset:(offset + batch_size), :]
        
        feed_dict = {train : batch_data, labels : batch_labels}        
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
    print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 50.770836
Minibatch accuracy: 3.9%
Minibatch loss at step 500: 0.525035
Minibatch accuracy: 100.0%
Minibatch loss at step 1000: 0.345277
Minibatch accuracy: 99.2%
Minibatch loss at step 1500: 0.344080
Minibatch accuracy: 99.2%
Minibatch loss at step 2000: 0.344074
Minibatch accuracy: 99.2%
Minibatch loss at step 2500: 0.344080
Minibatch accuracy: 99.2%
Minibatch loss at step 3000: 0.344086
Minibatch accuracy: 99.2%
Validation accuracy: 75.7%
Test accuracy: 83.1%


Typical overfitting. $99\%$ accuracy on train and much lower on valid & test.

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [8]:
d_h = 1024

graph = tf.Graph()
with graph.as_default():
    train = tf.placeholder(tf.float32, (batch_size, image_size * image_size))
    labels = tf.placeholder(tf.float32, (batch_size, num_labels))
    keep_prob = tf.placeholder_with_default(1., shape=())
    
    valid = tf.constant(valid_dataset)
    test = tf.constant(test_dataset)
    
    w1 = tf.Variable(tf.truncated_normal([image_size * image_size, d_h]))
    b1 = tf.Variable(tf.zeros([d_h]))
    
    w2 = tf.Variable(tf.truncated_normal([d_h, num_labels]))
    b2 = tf.Variable(tf.zeros([num_labels]))
    
    def model(data):        
        dense_relu = tf.nn.relu(tf.matmul(data, w1) + b1)
        dropout = tf.nn.dropout(dense_relu, keep_prob)
        return tf.matmul(dropout, w2) + b2  
    
    logits = model(train)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=labels))
    
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(valid))
    test_prediction = tf.nn.softmax(model(train))

Let's use keep_prob = 1 in _training_, so no dropout at all on tiny dataset.

In [9]:
nb_steps = 3001

In [10]:
def dropout(p):
    with tf.Session(graph=graph) as session:
        tf.global_variables_initializer().run()
        print('initialized')
        for step in range(nb_steps):
            # Pick an offset within the training data, which has been randomized.
            # Note: we could use better randomization across epochs.
            offset = (step * batch_size) % (tiny_labels.shape[0] - batch_size)
            # Generate a minibatch.
            batch_data = tiny_train[offset:(offset + batch_size), :]
            batch_labels = tiny_labels[offset:(offset + batch_size), :]
            # Prepare a dictionary telling the session where to feed the minibatch.
            # The key of the dictionary is the placeholder node of the graph to be fed,
            # and the value is the numpy array to feed to it.
            train_feed_dict = {train : batch_data, labels : batch_labels, keep_prob : p}
            _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=train_feed_dict)
            if (step % 500 == 0):
                print("Minibatch loss at step %d: %f" % (step, l))
                print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))

        test_feed_dict = {train : batch_data, labels : batch_labels, keep_prob : 1}
        print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(test_feed_dict), valid_labels))
        print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(test_feed_dict), test_labels))

In [11]:
dropout(1)

initialized
Minibatch loss at step 0: 274.158203
Minibatch accuracy: 14.8%
Minibatch loss at step 500: 0.000000
Minibatch accuracy: 100.0%
Minibatch loss at step 1000: 0.000000
Minibatch accuracy: 100.0%
Minibatch loss at step 1500: 0.000000
Minibatch accuracy: 100.0%
Minibatch loss at step 2000: 0.000000
Minibatch accuracy: 100.0%
Minibatch loss at step 2500: 0.000000
Minibatch accuracy: 100.0%
Minibatch loss at step 3000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 73.7%
Test accuracy: 81.2%


And now we are gonna look at the result with proper dropout :)

In [12]:
dropout(0.5)

initialized
Minibatch loss at step 0: 427.651245
Minibatch accuracy: 16.4%
Minibatch loss at step 500: 0.000000
Minibatch accuracy: 100.0%
Minibatch loss at step 1000: 0.006763
Minibatch accuracy: 99.2%
Minibatch loss at step 1500: 0.000000
Minibatch accuracy: 100.0%
Minibatch loss at step 2000: 0.000000
Minibatch accuracy: 100.0%
Minibatch loss at step 2500: 0.000000
Minibatch accuracy: 100.0%
Minibatch loss at step 3000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 77.7%
Test accuracy: 84.4%


Significantly better result on test!

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [64]:
d_h1 = 1024
d_h2 = 128

graph = tf.Graph()
with graph.as_default():
    train = tf.placeholder(tf.float32, (batch_size, image_size * image_size))
    labels = tf.placeholder(tf.float32, (batch_size, num_labels))
    keep_prob = tf.placeholder_with_default(1., shape=())
    valid = tf.constant(valid_dataset)
    test = tf.constant(test_dataset)
    
    global_step = tf.Variable(0, trainable=False)
    starter_learning_rate = 0.5
    learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step, 1000, 0.96)
    
    w1 = tf.Variable(tf.truncated_normal([image_size * image_size, d_h1]))
    b1 = tf.Variable(tf.zeros([d_h1]))
    
    w2 = tf.Variable(tf.truncated_normal([d_h1, d_h2]))
    b2 = tf.Variable(tf.zeros([d_h2]))  
    
    w3 = tf.Variable(tf.truncated_normal([d_h2, num_labels]))
    b3 = tf.Variable(tf.zeros([num_labels]))
    
    def model(data):        
        dense_relu = tf.nn.relu(tf.matmul(data, w1) + b1)
        dropout = tf.nn.dropout(dense_relu, keep_prob)        
        dense_tanh = tf.nn.tanh(tf.matmul(dropout, w2) + b2)
        dropout = tf.nn.dropout(dense_tanh, keep_prob)  
        return tf.matmul(dropout, w3) + b3
    
    logits = model(train)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=labels))
    
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
    
    train_prediction = tf.nn.softmax(logits)    
    valid_prediction = tf.nn.softmax(model(valid))    
    test_prediction = tf.nn.softmax(model(test))

In [65]:
nb_steps = 101337

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('initialized')
    for step in range(nb_steps):
        offset = (step * batch_size) % (train_dataset.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        train_feed_dict = {train : batch_data, labels : batch_labels, keep_prob : 0.5}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=train_feed_dict)
        if (step % 5000 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))

    test_feed_dict = {train : batch_data, labels : batch_labels, keep_prob : 1.}
    print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(test_feed_dict), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(test_feed_dict), test_labels))

initialized
Minibatch loss at step 0: 19.531572
Minibatch accuracy: 13.3%
Minibatch loss at step 5000: 0.889902
Minibatch accuracy: 68.8%
Minibatch loss at step 10000: 0.893476
Minibatch accuracy: 76.6%
Minibatch loss at step 15000: 0.948846
Minibatch accuracy: 74.2%
Minibatch loss at step 20000: 0.708825
Minibatch accuracy: 82.0%
Minibatch loss at step 25000: 0.975023
Minibatch accuracy: 73.4%
Minibatch loss at step 30000: 0.627755
Minibatch accuracy: 82.8%
Minibatch loss at step 35000: 0.832752
Minibatch accuracy: 75.0%
Minibatch loss at step 40000: 0.817461
Minibatch accuracy: 76.6%
Minibatch loss at step 45000: 0.620715
Minibatch accuracy: 82.0%
Minibatch loss at step 50000: 0.736896
Minibatch accuracy: 78.9%
Minibatch loss at step 55000: 0.911042
Minibatch accuracy: 75.0%
Minibatch loss at step 60000: 0.840913
Minibatch accuracy: 75.0%
Minibatch loss at step 65000: 0.845895
Minibatch accuracy: 74.2%
Minibatch loss at step 70000: 0.545467
Minibatch accuracy: 84.4%
Minibatch loss at

Not bad!